## Resellers Superstore

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import glob
from datetime import date

In [2]:
## There are the broad categories
url = ['http://master-resale-rights.com/personal-niches',
      'http://master-resale-rights.com/web-design',
      'http://master-resale-rights.com/webmastering',
      'http://master-resale-rights.com/videos',
      'http://master-resale-rights.com/audios',
      'http://master-resale-rights.com/business',
      'http://master-resale-rights.com/marketing']



## Get all the sub category links from those broad category links
sub_link = []
for lnk in url:
    r = requests.get(lnk)
    s = BeautifulSoup(r.content,'html.parser')

    for lnk in s.find_all('div',class_='catalog-product-sub-categories spacer clearfix'):
        for lnk1 in lnk.find_all('ul'):
            for lnk2 in lnk1.find_all('a'):
                sub_link.append(lnk2.get('href'))

In [3]:
# These are the sub category links
sub_link[0:10]

['http://master-resale-rights.com/personal-niches/animal-related',
 'http://master-resale-rights.com/personal-niches/computing',
 'http://master-resale-rights.com/personal-niches/family',
 'http://master-resale-rights.com/personal-niches/gaming',
 'http://master-resale-rights.com/personal-niches/health-beauty',
 'http://master-resale-rights.com/personal-niches/hobbies',
 'http://master-resale-rights.com/personal-niches/holiday-related',
 'http://master-resale-rights.com/personal-niches/home-and-garden',
 'http://master-resale-rights.com/personal-niches/personal-finance',
 'http://master-resale-rights.com/personal-niches/recipes-and-cooking']

In [4]:
def scrape(sub_url):
    
    # Variables to be extracted
    title = []
    title_link= []
    price = []
    
    # We iterate through 1 to 21 page for each sub category links
    for pg in range(1,22):
        r = requests.get(sub_url+f'-page-{pg}')
        s = BeautifulSoup(r.content,'html.parser')
        
        # Scrape title
        try:
            for i in s.find_all('h5',class_='catalog-product-title'):
                title.append(i.a.text)
        except:
            title.append('missing')
         
        # Scrape title link
        try:
            for i in s.find_all('h5',class_='catalog-product-title'):
                title_link.append(i.a.get('href'))
        except:
            title_link.append('missing')
        

        # Scrape price
        try:
            for i in s.find_all('span',class_='sale-price-amount'):
                price.append(i.text)
        except:
            price.append('missing')
            
    # Create a df with those extracted variables
    df = pd.DataFrame({'title':title,
                      'link':title_link,
                      'price':price})
    # Extract category
    df['cat'] = sub_url.split('/')[3]
    
    # Extract sub category
    df['sub_cat'] = sub_url.split('/')[4]
    
    return df.to_csv(f'{df.sub_cat.iloc[0]}.csv',index=False)

In [ ]:
## Iterate through all sub category links and extract data
for lnk in sub_link:
    try:
        scrape(lnk)
    except:
        pass

In [ ]:
# Concat all the csv files with name "reseller superstores"
today = date.today().strftime('%d_%b')
df = pd.concat([pd.read_csv(f, encoding='latin1') for f in glob.glob('*.csv')],sort=False).reset_index(drop=True)
df.to_csv(f'{today}_reseller_superstores.csv',index=False)